# Восстановление изображений
Старый файл с основными методами восстановления.

Время работы: 18с

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.signal import convolve2d
from numpy.fft import fftn, ifftn

from skimage import color, data, restoration

import numpy as np
from skimage import color, data, img_as_float, io

from functools import partial
from scipy.optimize import minimize

from util import *
from richardson_lucy import *

In [ ]:
astro = color.rgb2gray(data.astronaut())
prefix = "pic/deconv"

In [ ]:
def plot_triptih(images, titles=('Original', 'Distorted', 'Restored')):
    # Support method for plotting images (orignal, distorted, restored)
    n = len(images)
    m = len(titles)
    fig, ax = plt.subplots(nrows=1, ncols=n, figsize=(20, 10))
    plt.gray()
    for a in ax:
           a.axis('off')

    for i,I in enumerate(images):
        ax[i].imshow(I)
        if i<m:
            ax[i].set_title(titles[i])

    fig.subplots_adjust(wspace=0.02, hspace=0.02,
                        top=0.9, bottom=0.05, left=0, right=1)

    plt.show()

In [ ]:
shift = 15
psf = motion_blur_psf_my(shift, pi/3)
astro_blurred = convolve2d(astro, psf, 'same', 'symm')
astro_blurred += 1e-5 * np.random.standard_normal(astro_blurred.shape)#astro_blurred.std()
#astro_blurred = edgetaper(astro_blurred, gkern2(60, 21))
npArrayToPng(astro_blurred, "{0}-astro-shift10.png".format(prefix))

# Инверсная фильтрация

In [ ]:
plt.figure(figsize=(6,6))
plt.imshow(astro_blurred, 'gray')
plt.show()

In [ ]:
psf2 = np.zeros(astro_blurred.shape)
psf2[:psf.shape[0], :psf.shape[1]] = psf

In [ ]:
astro_blurred = edgetaper(astro_blurred, gkern2(60, 21))
astro_restored = ifftn(fftn(astro_blurred)/psf2otf(psf, astro_blurred.shape))
#astro_restored = ifftn(fftn(astro_blurred)/fftn(psf2))
astro_restored = np.clip(np.abs(astro_restored),0, 1)
npArrayToPng(astro_restored, "{0}-inverse-filter.png".format(prefix))

In [ ]:
print(compare_psnr_crop(astro, astro_restored))
plt.imshow(astro_restored, 'gray')
plt.show()

## Unsupervised wiener
box 10x10 filter

In [ ]:
shift = 15
psf = motion_blur_psf_my(shift, pi/3)
astro_distorted = convolve2d(astro, psf, 'same', 'symm')
astro_distorted += 0.001 * np.random.standard_normal(astro_distorted.shape)# * astro_distorted.std()
#astro_distorted = edgetaper(astro_distorted, gkern2(60, 21))

deconvolved, _ = restoration.unsupervised_wiener(edgetaper(astro_distorted, gkern2(60, 21)), psf)
deconvolved = np.clip(deconvolved, 0, 1)

plot_triptih((astro, astro_distorted, deconvolved),
            ('Data', 'Distorted', 'Self tuned restoration'))
npArrayToPng(astro_distorted, "{0}-shift15.png".format(prefix))
npArrayToPng(deconvolved, "{0}-shift15-restored-u-wiener.png".format(prefix))
print(compare_psnr_crop(astro, deconvolved))

In [ ]:
import skimage.data
from skimage import img_as_float
img = img_as_float(skimage.data.camera())
sigma = 0.1
img = img + sigma * np.random.standard_normal(img.shape)
sigma_hat = restoration.estimate_sigma(img, multichannel=False)
sigma, sigma_hat

## Wiener
box 10x10

In [ ]:
astro_distorted = convolve2d(astro, psf, 'same', 'symm')
astro_distorted += 0.1 * astro_distorted.std() * np.random.standard_normal(astro_distorted.shape)
#astro_distorted = edgetaper(astro_distorted, gkern2(60, 21))

deconvolved_img = restoration.wiener(edgetaper(astro_distorted, gkern2(60, 21)), psf, 1)
deconvolved_img = np.clip(deconvolved, 0, 1)
print(compare_psnr_crop(astro, deconvolved_img))
plot_triptih((astro, astro_distorted, deconvolved_img))

In [ ]:
astro = color.rgb2gray(data.astronaut())
N = 10
psf = np.ones((N, N)) / N**2
astro_distorted = convolve2d(astro, psf, 'same', 'symm')
astro_distorted += 0.1 * astro_distorted.std() * np.random.standard_normal(astro_distorted.shape)
#astro_distorted = edgetaper(astro_distorted, gkern2(60, 21))

#sigma_hat = restoration.estimate_sigma(I_noised, multichannel=False)

deconvolved_img = restoration.wiener(edgetaper(astro_distorted, gkern2(60, 21)), psf, 1)
print(compare_psnr_crop(astro, deconvolved_img))
plot_triptih((astro, astro_distorted, deconvolved_img),
             ('Original Data', 'Box {0}x{0} blurred'.format(N), 'Restoration using\nWiener'))

## Lucy-Richardson
box 10x10 filter

In [ ]:
shift = 15
psf = motion_blur_psf_my(shift, pi/3)
astro_distorted = convolve2d(astro, psf, 'same', 'symm')
astro_distorted += 0.001 * np.random.standard_normal(astro_distorted.shape)
#astro_distorted = edgetaper(astro_distorted, gkern2(60, 21))
# Add Noise to Image
astro_noisy = astro.copy()
astro_noisy += (np.random.poisson(lam=25, size=astro.shape) - 10) / 255.

# Restore Image using Richardson-Lucy algorithm
#richardson_lucy_matlab(astro_noisy, psf, iterations=50, dam)
deconvolved_RL = richardson_lucy_matlab(edgetaper(astro_distorted, gkern2(30, 11)), psf, iterations=30)
print(compare_psnr_crop(astro, deconvolved_RL['image']))
plot_triptih((astro, astro_noisy, deconvolved_RL['image']),
             ('Original Data', 'Box {0}x{0} blurred'.format(N), 'Restoration using\nRichardson-Lucy'))

In [ ]:
astro_distorted

Motion blur (x+=20) filter

In [ ]:
shift = 15
psf = motion_blur_psf_my(shift, pi/3)
astro_distorted = convolve2d(astro, psf, 'same', 'symm')
astro_distorted += 0.001 * np.random.standard_normal(astro_distorted.shape)
#astro_distorted = edgetaper(astro_distorted, gkern2(60, 21))

deconvolved_RL = richardson_lucy_matlab(edgetaper(astro_distorted, gkern2(30, 11)), psf, iterations=50)
print(compare_psnr_crop(astro, deconvolved_RL['image']))
plot_triptih((astro, astro_blurred, deconvolved_RL['image']),
             ('Original Data', 'Motion blurred {0} pixels'.format(N), 'Restoration using\nRichardson-Lucy'))

In [ ]:
def gauss2dKernel(sigma, size=3):
    # First a 1-D  Gaussian
    t = np.linspace(-(size-1)/2., (size-1)/2., size)
    bump = np.exp(-t**2/(2*sigma**2))
    return bump[:, np.newaxis] * bump[np.newaxis, :] / (2*np.pi*sigma**2)

In [ ]:
I = color.rgb2gray(data.astronaut())

psf_mv = gauss2dKernel(3, 20)
I_moved = convolve2d(I, psf_mv, 'same')

# Add Noise to Image
I_noised = I_moved.copy()
I_noised += (np.random.poisson(lam=25, size=I_noised.shape) - 10) / 255.
#I_noised = edgetaper(I_noised, gkern2(60, 21))

deconvolved_RL = richardson_lucy_matlab(edgetaper(I_noised, gkern2(60, 21)), psf_mv, iterations=30)
print(compare_psnr_crop(astro, deconvolved_RL['image']))
plot_triptih((I, I_noised, deconvolved_RL['image']),
             ('Original Data', 'Gauss blurred', 'Restoration using\nRichardson-Lucy'))

# Спектр

In [ ]:
psf = motion_blur_psf_my(40, pi/6)
astro_blurred = convolve2d(astro, psf, 'same', 'symm')
#astro_blurred += 5e-2 * np.random.standard_normal(astro_blurred.shape)
plt.imshow(astro_blurred)
plt.show()

In [ ]:
astro_blurred_spectre = np.clip(np.abs(fftn(astro_blurred)), 0, 2)
astro_blurred_spectre = np.fft.fftshift(astro_blurred_spectre)
plt.imshow(astro_blurred_spectre, 'gray')
plt.show()

In [ ]:
npArrayToPng(astro, 'pic/astro.png')
npArrayToPng(astro_blurred, '{0}-astro-blurred-shift40.png'.format(prefix))
npArrayToPng(astro_blurred_spectre, '{0}-astro-spectre-shift40.png'.format(prefix))

# Кепстр

In [ ]:
N,M = astro_blurred.shape
# ( ifft2 (100* log (1+ abs ( fft2 ( I ) ) ) ) ) ;
K = np.fft.ifftn(100*np.log(1+np.abs(np.fft.fftn(astro_blurred))))
K_shift = np.fft.fftshift(K)

nn = np.argmin(np.real(K_shift))
n, m, = K_shift.shape
r, c = np.unravel_index(np.argmin(K_shift, axis=None), K_shift.shape)
x0 = [r - n//2, c - m//2]
print(x0)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.clip(np.real(K_shift),-2,1))#, vmin=noisy.min(), vmax=noisy.max())
npArrayToPng(np.clip(np.real(K_shift),-2,1), '{0}-kepstr-shift40-pi6.png'.format(prefix))
plt.plot(n//2 + x0[1], m//2 + x0[0], 'ro')
plt.title('Кепстр изображения с выделенным минмиумом')
#plt.savefig('pic/{0}-kepstr.png'.format(prefix))
plt.show()

# С шумом

In [ ]:
astro_blurred = convolve2d(astro, psf, 'same')
astro_blurred += 1e-1 * np.random.standard_normal(astro_blurred.shape)
N,M = astro_blurred.shape

K = np.fft.ifftn(100*np.log(1+np.abs(np.fft.fftn(astro_blurred))))
K_shift = np.fft.fftshift(K)

nn = np.argmin(np.real(K_shift))
n, m, = K_shift.shape
r, c = np.unravel_index(np.argmin(K_shift, axis=None), K_shift.shape)
x0 = [r - n//2, c - m//2]
print(x0)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.clip(np.real(K_shift),-2,1))#, vmin=noisy.min(), vmax=noisy.max())
npArrayToPng(np.clip(np.real(K_shift),-2,1), '{0}-kepstr-noised-shift40-pi6.png'.format(prefix))
plt.plot(n//2 + x0[1], m//2 + x0[0], 'ro')
plt.title('Кепстр изображения с выделенным минмиумом')
#plt.savefig('pic/{0}-kepstr.png'.format(prefix))
plt.show()